In [1]:
import json

import pandas as pd
from tqdm import tqdm
import tweepy
from dotenv import load_dotenv
import os

from tweepy import TweepyException

load_dotenv()  # load environment variables from .env

True

In [2]:
DATA_PATH = 'data'
YEARS = [2016, 2020]

In [16]:
for year in YEARS:
    with open(os.path.join(DATA_PATH, f'polls-{year}.json')) as f:
        polls = json.load(f)
    author_ids = [poll['meta']['author_id'] for poll in filter(lambda x: x['type']=='twitter', polls)]
    with open(os.path.join(DATA_PATH, f'polls_{year}_author_ids.json'), 'w+') as f:
        json.dump(author_ids, f)

In [17]:
auth = tweepy.OAuth2BearerHandler(
    bearer_token= os.environ['BEARER'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [29]:
exception_users = dict()

In [30]:

for year in YEARS:
    out_path = os.path.join(DATA_PATH, f'follower_lists_{year}.jsonl')
    existing_users = set()
    if os.path.exists(out_path):
        with open(out_path) as f:
            existing_users.update(k for l in f for k in json.loads(l))
            print(f'found {len(existing_users)} scraped users')

    with open(os.path.join(DATA_PATH, f'polls_{year}_author_ids.json')) as f:
        author_ids = [id for id in json.load(f) if id not in existing_users]

    with open(os.path.join(DATA_PATH, f'follower_lists_{year}.jsonl'), 'a+', encoding='utf8') as f:
        for author_id in tqdm(author_ids, desc='checking followers of poll authors'):
            follower_ids = []
            try:
                for page in tweepy.Cursor(api.get_follower_ids, user_id = author_id, count = 5000).pages():
                    follower_ids.extend(page)
                f.write(json.dumps({author_id:follower_ids}) + '\n')
            except TweepyException as e:
                print(e)
                print(author_id)
                exception_users[author_id]=e


found 333 scraped users


checking followers of poll authors:   6%|▌         | 1/17 [00:00<00:03,  4.91it/s]

404 Not Found
34 - Sorry, that page does not exist.
2515437133


checking followers of poll authors:  18%|█▊        | 3/17 [00:00<00:02,  4.86it/s]

401 Unauthorized
Not authorized.
155939890


...

404 Not Found
34 - Sorry, that page does not exist.
1216703337317457921
401 Unauthorized
Not authorized.
63998604


checking followers of poll authors:  98%|█████████▊| 54/55 [45:03<00:07,  7.85s/it]

401 Unauthorized
Not authorized.
1110075862328717312


checking followers of poll authors: 100%|██████████| 55/55 [45:03<00:00, 49.16s/it]

404 Not Found
34 - Sorry, that page does not exist.
1292939157375987712


In [31]:
len(exception_users)

60

In [41]:
with open(os.path.join(DATA_PATH, 'authors_missing.jsonl'), 'w+') as f:
    for k, v in exception_users.items():
        f.write(json.dumps({k:v.api_errors[0]})+'\n')

In [42]:
for year in YEARS:
    out_path = os.path.join(DATA_PATH, f'follower_lists_{year}.jsonl')
    existing_users = set()
    if os.path.exists(out_path):
        with open(out_path) as f:
            existing_users.update(k for l in f for k in json.loads(l))
            print(f'found {len(existing_users)} scraped users')

    with open(os.path.join(DATA_PATH, f'polls_{year}_author_ids.json')) as f:
        author_ids = [id for id in json.load(f) if id not in existing_users]
        print(len(author_ids))
        print(len([i for i in author_ids if i not in exception_users]))

found 334 scraped users
16
0
found 636 scraped users
55
0


In [54]:
import pandas as pd

fnames = ['senators-accounts-1.csv',
'representatives-accounts-1.csv',
'congress116-house-accounts.csv',
'congress116-senate-accounts.csv',]
congress_ids = set(pd.concat([pd.read_csv(os.path.join(DATA_PATH, fname)) for fname in fnames]).Uid.unique())
len(congress_ids)

1050

In [55]:
congress_ids

{803694179079458816,
 816683274076614656,
 818554054309715969,
 817076257770835968,
 811313565760163844,
 811986281177772032,
 816157667882373120,
 2974648323,
 806583915012046854,
 818472418620608512,
 816298918468259841,
 815966620300480514,
 816652616625168388,
 798973032362606600,
 161411080,
 442824717,
 21157904,
 855240223432769538,
 829061809135030272,
 304138251,
 193441812,
 4228409359,
 1080574793630527505,
 158890005,
 1058717720,
 1129029661,
 242376736,
 42481696,
 550131748,
 385429543,
 5558312,
 2976606250,
 117501995,
 43986986,
 2933760046,
 303861808,
 404355121,
 4827594804,
 188403766,
 22044727,
 237862972,
 231510077,
 1058807868,
 218292287,
 153507902,
 158470209,
 970207298,
 3018670151,
 816303263586914304,
 721127537674489856,
 233949261,
 241207373,
 3026622545,
 1058345042,
 22812754,
 27775884,
 239548513,
 1060487274,
 233693291,
 377534571,
 435331179,
 291756142,
 3170031728,
 2973870195,
 223166587,
 231108733,
 1080307235350241280,
 8160304247785431